In [2]:
import torch
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data

In [3]:
x = torch.tensor([[2,1], [5, 6], [3, 7], [12, 0]], dtype=torch.float)
y = torch.tensor([0, 1, 0, 1], dtype=torch.float)

In [4]:
edge_index = torch.tensor([[0, 1, 2, 0, 3], [1, 0, 1, 3, 2]], dtype=torch.long)

In [5]:
edge_index

tensor([[0, 1, 2, 0, 3],
        [1, 0, 1, 3, 2]])

In [6]:
data = Data(x=x, y=y, edge_index=edge_index)
print(data)
device = torch.device('cuda')

Data(x=[4, 2], edge_index=[2, 5], y=[4])


In [7]:
data1 = [data, data]
data2 = []
for data0 in data1:
    data0.to(device)
    data2.append(data0)

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [8]:
data2[0]

IndexError: list index out of range

### Dataset

In [9]:
￼Untitled.

SyntaxError: invalid character in identifier (37248126.py, line 1)

### DataLoader

In [10]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv(r'/home/huihai/Downloads/yoochoose-data/yoochoose-clicks.dat', header=None)

FileNotFoundError: [Errno 2] No such file or directory: '/home/huihai/Downloads/yoochoose-data/yoochoose-clicks.dat'

In [12]:
df.columns=["session_id", "timestamp", "item_id", "category"]

NameError: name 'df' is not defined

In [11]:
df

,session_id,timestamp,item_id,category
0,1,2014-04-07T10:51:09.277Z,214536502,0
1,1,2014-04-07T10:54:09.868Z,214536500,0
2,1,2014-04-07T10:54:46.998Z,214536506,0
3,1,2014-04-07T10:57:00.306Z,214577561,0
4,2,2014-04-07T13:56:37.614Z,214662742,0
...,...,...,...,...
33003939,11299809,2014-09-25T09:33:22.412Z,214819412,S
33003940,11299809,2014-09-25T09:43:52.821Z,214830939,S
33003941,11299811,2014-09-24T19:02:09.741Z,214854855,S
33003942,11299811,2014-09-24T19:02:11.894Z,214854838,S


In [13]:
buy_df = pd.read_csv(r"/home/huihai/Downloads/yoochoose-data/yoochoose-buys.dat")
buy_df.columns=["session_id", "time_stamp", "item_id", "price", "quantity"]

FileNotFoundError: [Errno 2] No such file or directory: '/home/huihai/Downloads/yoochoose-data/yoochoose-buys.dat'

In [13]:
item_encoder = LabelEncoder()

In [14]:
df['item_id'] = item_encoder.fit_transform(df.item_id)

In [15]:
df.head()

,session_id,timestamp,item_id,category
0,1,2014-04-07T10:51:09.277Z,2053,0
1,1,2014-04-07T10:54:09.868Z,2052,0
2,1,2014-04-07T10:54:46.998Z,2054,0
3,1,2014-04-07T10:57:00.306Z,9876,0
4,2,2014-04-07T13:56:37.614Z,19448,0


In [16]:
df['label'] = df.session_id.isin(buy_df.session_id)
df.head()

,session_id,timestamp,item_id,category,label
0,1,2014-04-07T10:51:09.277Z,2053,0,False
1,1,2014-04-07T10:54:09.868Z,2052,0,False
2,1,2014-04-07T10:54:46.998Z,2054,0,False
3,1,2014-04-07T10:57:00.306Z,9876,0,False
4,2,2014-04-07T13:56:37.614Z,19448,0,False


In [17]:
sampled_session_id = np.random.choice(df.session_id.unique(), 10000, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()

session_id    10000
timestamp     34981
item_id        7921
category         47
label             2
dtype: int64

### Dataset Construction

In [18]:
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
from torch_geometric.data import Dataset

In [19]:
class YooChooseBinaryDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseBinaryDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return ['yoochoose_click_binary_1M_sess.dataset']
    def download(self):
        pass
    def process(self):
        data_list = []
        
        #process by session_id
        grouped = df.groupby("session_id")
        for session_id, group in tqdm(grouped):
            sess_item_id = LabelEncoder().fit_transform(group.item_id)
            group = group.reset_index(drop=True)
            group["sess_item_id"] = sess_item_id
            node_features = group.loc[group.session_id==session_id, ['sess_item_id', 'item_id']].sort_values('sess_item_id').item_id.drop_duplicates().values
            node_features = torch.LongTensor(node_features).unsqueeze(1)
            
            target_nodes = group.sess_item_id.values[1:]
            source_nodes = group.sess_item_id.values[:-1]
            edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)
            
            x = node_features
            y = torch.FloatTensor([group.label.values[0]])
            y = [y, y]

            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


In [20]:
dataset = YooChooseBinaryDataset(root='../datasettest')

Processing...
  0%|                                                                                                                                                                            | 0/10000 [00:00<?, ?it/s]/tmp/ipykernel_112749/226299429.py:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1666643003845/work/torch/csrc/utils/tensor_new.cpp:230.)
  edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group

  1%|▊                                                                                                                                                                 | 52/10000 [00:00<00:19, 514.71it/s]/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/2262994

/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
  1%|█▊                                                                                                                                                               | 109/10000 [00:00<00:18, 546.63it/s]/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/2262994

  2%|██▋                                                                                                                                                              | 167/10000 [00:00<00:17, 561.70it/s]/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/2262994

/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
  2%|███▋                                                                                                                                                             | 230/10000 [00:00<00:16, 586.36it/s]/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/2262994

...

/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
 98%|███████████████████████████████████

/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: D

/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: D

/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: D

/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([group.label.values[0]])
/tmp/ipykernel_112749/226299429.py:31: D

In [21]:
dataset = dataset.shuffle()
train_dataset = dataset[:8000]
val_dataset = dataset[8000:9000]
test_dataset = dataset[9000:]
len(train_dataset), len(val_dataset), len(test_dataset)

(8000, 1000, 1000)

In [22]:
from torch_geometric.data import DataLoader
batch_size= 1024
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

/home/huihai/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [23]:
num_items = df.item_id.max() +1 
num_items 

52707

In [24]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops
class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max') #  "Max" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]

        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]


        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

In [82]:
embed_dim = 128
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = SAGEConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)        

        x = F.relu(self.conv1(x, edge_index))

        x, edge_index, _, batch, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x

In [83]:
device = torch.device('cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()

In [87]:
def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        print(output)
        print(data)
        label = data.y[0].to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)

In [88]:
from sklearn.metrics import roc_auc_score
def evaluate(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y[0].detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    predictions = np.hstack(predictions)
    labels = np.hstack(labels)
    
    return roc_auc_score(labels, predictions)

In [89]:
for epoch in range(1):
    loss = train()
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)    
    test_acc = evaluate(test_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
          format(epoch, loss, train_acc, val_acc, test_acc))

ValueError: too many values to unpack (expected 5)